# Submission Guide



*   After completion of implementation, you should run all the cells
*   Submit your ipython notebook in 'ipynb' format
    * **Do not remove your output results from every cells**
*   File name format: lab1_studentID_name.
ipynb
    * Ex) lab1_20233809_MinsungHwang.ipynb
*   Submission due: March 27th by 10:00AM

    * **We do not accept late submissions**



# Load Dataset

In [87]:
from google.colab import files
f = files.upload()

Saving edges.txt to edges (2).txt
Saving nodes.txt to nodes (2).txt


In [88]:
edges = f['edges.txt'].decode('utf-8').strip().split('\n')
edgeList = [(edge.split()[0], edge.split()[1]) for edge in edges]

nodeList = f['nodes.txt'].decode('utf-8').strip().split('\n')
node2id = dict(zip(nodeList, range(len(nodeList))))

# Computing Personalized PageRanks

In [89]:
class Node:
    def __init__(self, id):
        self.id = id
        self.inNode = []
        self.outNode = []
        self.outDegree = 0

        self.pagerank = 0
        self.pagerankNext = 0
        self.personalized = 0
    
    # TODO : Compute Personalized PageRank of a single node
    def aggregate_pagerank(self, alpha):
        # sum_of_inNode_pagerank =0
        # for node in self.inNode:
        #   sum_of_inNode_pagerank += node.pagerank
        # outgoing = alpha *sum((node.pagerank/node.outDegree) for node in self.inNode)
        
        self.pagerankNext = alpha *sum((node.pagerank/node.outDegree) for node in self.inNode) + self.personalized
        

    def update_pagerank(self):
      self.pagerank = self.pagerankNext

In [90]:
class Graph:
    def __init__(self, nodeList, edgeList):
        self.nodeList = nodeList
        self.edgeList = edgeList
        self.numNodes = len(self.nodeList)
        self._build_graph()
    
    def _build_graph(self):
        self.nodes = {}
        for id in self.nodeList:
            node = Node(id)
            self.nodes[id] = node
        
        for edge in self.edgeList:
            headID = edge[0]
            tailID = edge[1]

            self.nodes[headID].outNode.append(self.nodes[tailID])
            self.nodes[tailID].inNode.append(self.nodes[headID])

            self.nodes[headID].outDegree += 1

In [91]:
def compute_PageRank(nodeList, edgeList, maxIters, alpha, tolerance, personalize = None):
    # TODO : Compute the Personalized PageRank with the predefined list #
    graph = Graph(nodeList, edgeList)
    if personalize is None:
      personalize = graph.nodeList
    norm = len(personalize)
    for node in graph.nodes.values():
      if node.id in personalize:
        node.pagerank = (1-alpha) /norm
        node.personalized = (1-alpha) /norm


    
    for iter in range(maxIters):
        # TODO : Implement one iteration
        # Calculate L_infty norm to check convergence
        prevPageranks = [node.pagerank for node in graph.nodes.values()]

        for node in graph.nodes.values():
          node.aggregate_pagerank(alpha)
        for node in graph.nodes.values():
          node.update_pagerank()
        
        currPageranks = [node.pagerank for node in graph.nodes.values()]
        #error term check
        error = max(abs(prevPagerank - currPagerank) for prevPagerank, currPagerank in zip(prevPageranks, currPageranks))
        # print(error)
        if error < tolerance:
          print("# of iterations: ", iter)
          break
        
    else:
      print("Max iteration reached: ", maxIters)

    pageranks = [node.pagerank for node in graph.nodes.values()]
    pageranks = dict(zip(graph.nodeList, pageranks))
    
    return pageranks

In [92]:
from prettytable import PrettyTable

def print_PageRank_top10(pageranks, name='PageRank'):
    pagerankSorted = sorted(pageranks.items(), reverse=True, key=lambda x: x[1])[:10]

    table = PrettyTable(field_names = ['Node ID', name])
    for id, score in pagerankSorted[:10]:
        table.add_row([id, round(score, 4)])
    print(table)

In [93]:
pageranks = compute_PageRank(nodeList, edgeList, 10000, 0.85, 1e-8)

print_PageRank_top10(pageranks)

# of iterations:  84
+----------------------------+----------+
|          Node ID           | PageRank |
+----------------------------+----------+
| stateorprovince:california |  0.032   |
|        city:florida        |  0.0154  |
|        plant:trees         |  0.0146  |
| personmexico:ryan_whitney  |  0.0131  |
|   stateorprovince:texas    |  0.0116  |
|        country:usa         |  0.0098  |
| sportsteam:ncaa_youth_kids |  0.0077  |
|      vegetable:pepper      |  0.0074  |
|  profession:professionals  |  0.0073  |
|     country:countries      |  0.0072  |
+----------------------------+----------+


In [94]:
# Set the predefined set in the form of a list
personalizeList = ['politicianus:joe_biden', 'politicianus:biden', 'politicianus:senator_biden']

personalizedPageranks = compute_PageRank(nodeList, edgeList, 10000, 0.85, 1e-8, personalize = personalizeList)

print_PageRank_top10(personalizedPageranks, name='PPR')

# of iterations:  85
+----------------------------+--------+
|          Node ID           |  PPR   |
+----------------------------+--------+
|     politicianus:biden     | 0.0569 |
|     politician:clinton     | 0.0529 |
|   politicianus:joe_biden   | 0.0519 |
| politicianus:senator_biden | 0.0506 |
|     politicianus:palin     | 0.0301 |
| stateorprovince:california | 0.0274 |
|   politicaloffice:office   | 0.0172 |
|      politician:obama      | 0.0161 |
|    politicianus:mccain     | 0.016  |
| politicianus:barack_obama  | 0.0141 |
+----------------------------+--------+


# Computing PageRank with Sparse Matrix

In [95]:
import numpy as np
from scipy.sparse import coo_matrix

def compute_PageRank_with_sparse_matrix(nodeList, edgeList, node2id, maxIters, alpha, tolerance, personalize = None):
    # TODO : Compute the Personalized PageRank with the predefined list using sparse matrix-vector multiplication
    numNodes = len(nodeList)

    newEdgeList = [(node2id[edge[0]], node2id[edge[1]]) for edge in edgeList]
    rows, cols, data = zip(*[(edge[0], edge[1], 1) for edge in newEdgeList])
    adjSparse = coo_matrix((data, (rows,cols)), shape = (numNodes, numNodes), dtype = float)

    D = np.array(adjSparse.sum(axis=1), dtype = float)
    D[D!=0] = 1.0/D[D!=0]
    P = adjSparse.multiply(D)
    PT = P.transpose()

    if personalize is None:
      personalizeVector = np.ones((numNodes, 1), dtype = float)
    else:
      personalizeVector = np.array([1 if node in personalize else 0 for node in nodeList], dtype = float).reshape((numNodes, 1))

    personalizeVector /= np.sum(personalizeVector)
    personalizeVector *= (1-alpha)
    pageranks = np.copy(personalizeVector)


    for iter in range(maxIters):
        # TODO : Implement power iteration
        # Calculate L_infty norm to check convergence
        prevPageranks = pageranks
        pageranks = alpha * PT @ pageranks + personalizeVector
        error = max(np.absolute(prevPageranks - pageranks))

        if error< tolerance:
          print("# of iterations: ", iter)
          break
    else:
        print("Max iteration reached: ", maxIters)
    pageranks = np.asarray(pageranks)
    pageranks = dict(zip(nodeList, pageranks.squeeze().tolist()))
    return pageranks

In [96]:
pageranks = compute_PageRank_with_sparse_matrix(nodeList, edgeList, node2id, 10000, 0.85, 1e-8)
print_PageRank_top10(pageranks)

# of iterations:  84
+----------------------------+----------+
|          Node ID           | PageRank |
+----------------------------+----------+
| stateorprovince:california |  0.032   |
|        city:florida        |  0.0154  |
|        plant:trees         |  0.0146  |
| personmexico:ryan_whitney  |  0.0131  |
|   stateorprovince:texas    |  0.0116  |
|        country:usa         |  0.0098  |
| sportsteam:ncaa_youth_kids |  0.0077  |
|      vegetable:pepper      |  0.0074  |
|  profession:professionals  |  0.0073  |
|     country:countries      |  0.0072  |
+----------------------------+----------+


In [97]:
personalizedPageranks = compute_PageRank_with_sparse_matrix(nodeList, edgeList, node2id, 10000, 0.85, 1e-8, personalize = personalizeList)
print_PageRank_top10(personalizedPageranks, name = 'PPR')

# of iterations:  85
+----------------------------+--------+
|          Node ID           |  PPR   |
+----------------------------+--------+
|     politicianus:biden     | 0.0569 |
|     politician:clinton     | 0.0529 |
|   politicianus:joe_biden   | 0.0519 |
| politicianus:senator_biden | 0.0506 |
|     politicianus:palin     | 0.0301 |
| stateorprovince:california | 0.0274 |
|   politicaloffice:office   | 0.0172 |
|      politician:obama      | 0.0161 |
|    politicianus:mccain     | 0.016  |
| politicianus:barack_obama  | 0.0141 |
+----------------------------+--------+
